In [ ]:

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# PyCaret clustering imports
from pycaret.clustering import *

# Imports adicionales para análisis
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.metrics.pairwise import pairwise_distances
import time
import logging

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('clustering_medicamentos.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

print("🔬 SISTEMA DE HOMOLOGACIÓN AUTOMÁTICA DE MEDICAMENTOS")
print("="*70)
print("📊 Objetivo: Clustering para recomendación de medicamentos similares")
print("🎯 Estrategia: Entrenar con todo, recomendar solo válidos")
print("🔝 Output: Top 5 medicamentos similares por CUM inválido")
print("="*70)


In [ ]:
logger.info("Cargando datasets...")

# Rutas de los archivos
path_original = Path("./data/medicamentos_train_original.parquet")
path_encoded = Path("./data/medicamentos_train_preprocesados.parquet")

# Verificar que existen los archivos
if not path_original.exists():
    raise FileNotFoundError(f"No se encontró: {path_original}")
if not path_encoded.exists():
    raise FileNotFoundError(f"No se encontró: {path_encoded}")

# Cargar datasets
df_original = pd.read_parquet(path_original)
df_encoded = pd.read_parquet(path_encoded)

print(f"📁 Dataset Original cargado: {df_original.shape}")
print(f"📁 Dataset Encodificado cargado: {df_encoded.shape}")
print()

# Información básica de los datasets
logger.info("Información de datasets:")
logger.info(f"Original - Shape: {df_original.shape}, Memory: {df_original.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
logger.info(f"Encodificado - Shape: {df_encoded.shape}, Memory: {df_encoded.memory_usage(deep=True).sum() / 1024**2:.1f} MB")


In [ ]:
# =============================================================================
# PASO 3: ANÁLISIS EXPLORATORIO RÁPIDO
# =============================================================================

print("🔍 ANÁLISIS EXPLORATORIO DE AMBOS DATASETS")
print("="*50)

# Dataset Original
print("📋 DATASET ORIGINAL:")
print(f"   - Columnas: {list(df_original.columns)}")
print(f"   - Tipos de datos: {df_original.dtypes.value_counts().to_dict()}")
print(f"   - Valores nulos: {df_original.isnull().sum().sum()}")
print()

# Dataset Encodificado  
print("📋 DATASET ENCODIFICADO:")
print(f"   - Columnas: {df_encoded.shape[1]} columnas")
print(f"   - Tipos de datos: {df_encoded.dtypes.value_counts().to_dict()}")
print(f"   - Valores nulos: {df_encoded.isnull().sum().sum()}")
print()

# Verificar columnas de validez en ambos datasets
validez_cols = ['ESTADO REGISTRO', 'ESTADO CUM', 'MUESTRA MÉDICA']
print("🔍 VERIFICACIÓN DE COLUMNAS DE VALIDEZ:")

# Inicializar variable para evitar errores
validos_orig = 0

# Verificar si todas las columnas están presentes
all_cols_present = all(col in df_original.columns for col in validez_cols)

if all_cols_present:
    validos_orig = len(df_original[
        (df_original['ESTADO REGISTRO'] == 'Vigente') & 
        (df_original['ESTADO CUM'] == 'Activo') & 
        (df_original['MUESTRA MÉDICA'] == 'No')
    ])

for col in validez_cols:
    if col in df_original.columns:
        print(f"   ✅ {col} presente en dataset original")
    else:
        print(f"   ❌ {col} NO presente en dataset original")

print(f"   📊 Medicamentos válidos en original: {validos_orig:,}")

# Para dataset encodificado, buscar columnas de validez
validez_encoded = []
for col in df_encoded.columns:
    if any(v.lower() in col.lower() for v in ['estado', 'muestra']):
        validez_encoded.append(col)

print(f"   📊 Columnas de validez en encodificado: {validez_encoded}")

In [ ]:
# =============================================================================
# PASO 9: PREPARACIÓN PARA CLUSTERING - DATASET ENCODIFICADO
# =============================================================================

print("\n🎯 PREPARACIÓN PARA CLUSTERING - DATASET ENCODIFICADO")
print("="*57)

# El dataset encodificado ya debe tener las columnas de validez encodificadas
print(f"📊 Dataset encodificado shape: {df_encoded.shape}")
print(f"📋 Tipos de datos: {df_encoded.dtypes.value_counts().to_dict()}")

# Para dataset encodificado, necesitamos identificar las columnas de validez
# Asumiendo que ya fueron encodificadas, buscar patrones
print("🔍 Buscando columnas de validez en dataset encodificado...")

# Buscar columnas relacionadas con validez (pueden estar encodificadas)
possible_validity_cols = [col for col in df_encoded.columns 
                         if any(term in col.lower() for term in ['estado', 'muestra', 'vigente', 'activo'])]

print(f"📋 Posibles columnas de validez encontradas: {possible_validity_cols}")

# Usar TODO el dataset encodificado - TU ya hiciste el trabajo de selección
df_clustering_encoded = df_encoded.copy()

print(f"📊 Columnas para clustering encodificado: {df_clustering_encoded.shape[1]}")


In [ ]:
# =============================================================================
# PASO 10: CONFIGURACIÓN DE PYCARET CLUSTERING - DATASET ENCODIFICADO
# =============================================================================

print("\n🚀 CONFIGURACIÓN DE PYCARET CLUSTERING - DATASET ENCODIFICADO")
print("="*62)

logger.info("Iniciando setup de PyCaret para dataset encodificado...")

try:
    start_time = time.time()
    
    # Para dataset encodificado, setup simple como el ejemplo de clase
    cluster_setup_encoded = setup(
        data=df_clustering_encoded,
        session_id=123,                   # Para reproducibilidad
        normalize=False,                  # ❌ NO normalizar - ya está hecho
        transformation=False,             # ❌ NO transformar - ya está hecho  
        pca=False,                        # ❌ NO aplicar PCA - usar todas las features
        remove_multicollinearity=False,   # ❌ NO remover multicolinealidad - ya se manejó
        remove_outliers=False,            # ❌ NO remover outliers - conservar datos
        preprocess=False,                 # ❌ NO hacer preprocessing adicional
        use_gpu=True,                     # Usar GPU si está disponible
        
        
    )
    
    setup_time_encoded = time.time() - start_time
    logger.info(f"Setup encodificado completado en {setup_time_encoded:.2f} segundos")
    print(f"✅ Setup PyCaret completado para dataset encodificado ({setup_time_encoded:.2f}s)")
    
except Exception as e:
    logger.error(f"Error en setup encodificado: {str(e)}")
    print(f"❌ Error en setup encodificado: {str(e)}")


In [ ]:
# =============================================================================
# COMPARACIÓN COMPLETA DE TODOS LOS ALGORITMOS DISPONIBLES
# =============================================================================

from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import time

#  algoritmos de PyCaret
algoritmos_clustering = [
    # si 'kmeans',      # K-Means Clustering
    #'ap',          # Affinity Propagation
    #'meanshift',   # Mean shift Clustering
    # 'sc',          # Spectral Clustering
    # 'hclust',      # Agglomerative Clustering
    'dbscan',      # Density-Based Spatial Clustering
    'optics',      # OPTICS Clustering
    'birch',       # Birch Clustering
    'kmodes'       # K-Modes Clustering
]


# Diccionario para almacenar resultados
resultados_comparacion = {}
metricas_comparacion = []

print("🔬 COMPARACIÓN COMPLETA DE TODOS LOS ALGORITMOS DE CLUSTERING")
print("="*65)
print(f"🎯 Algoritmos a probar: {len(algoritmos_clustering)}")
for i, algo in enumerate(algoritmos_clustering, 1):
    print(f"   {i}. {algo.upper()}")
print("="*65)

for i, algoritmo in enumerate(algoritmos_clustering, 1):
    print(f"\n🧪 Probando {i}/{len(algoritmos_clustering)}: {algoritmo.upper()}")
    print("-" * 50)
    
    try:
        # Medir tiempo de entrenamiento
        start_time = time.time()
        
        # Crear modelo con parámetros específicos según algoritmo
        if algoritmo == 'kmeans':
            # K-means: especificar número de clusters
            modelo = create_model(algoritmo, num_clusters=20)
        elif algoritmo == 'sc':
            # Spectral: especificar número de clusters
            modelo = create_model(algoritmo, num_clusters=15)
        elif algoritmo == 'hclust':
            # Hierarchical: especificar número de clusters
            modelo = create_model(algoritmo, num_clusters=25)
        elif algoritmo == 'birch':
            # BIRCH: especificar número de clusters
            modelo = create_model(algoritmo, num_clusters=20)
        elif algoritmo == 'kmodes':
            # K-modes: para variables categóricas
            modelo = create_model(algoritmo, num_clusters=15)
        else:
            # Para AP, MEANSHIFT, DBSCAN, OPTICS: automático
            modelo = create_model(algoritmo)
        
        training_time = time.time() - start_time
        
        # Asignar clusters
        print(f"   ⏳ Asignando clusters...")
        assign_start = time.time()
        resultados = assign_model(modelo)
        assign_time = time.time() - assign_start
        
        # Verificar resultados
        n_clusters = resultados['Cluster'].nunique()
        n_noise = (resultados['Cluster'] == -1).sum() if -1 in resultados['Cluster'].values else 0
        total_time = training_time + assign_time
        
        print(f"   ✅ Completado en {total_time:.2f}s (Train: {training_time:.2f}s, Assign: {assign_time:.2f}s)")
        print(f"   📊 Clusters generados: {n_clusters}")
        print(f"   🔍 Puntos de ruido: {n_noise} ({n_noise/len(resultados)*100:.1f}%)")
        
        # Calcular métricas de evaluación
        if n_clusters > 1:
            try:
                print(f"   📈 Calculando métricas...")
                
                # Obtener datos originales
                X = get_config('X_train')
                labels = resultados['Cluster'].values
                
                # Filtrar ruido para métricas (si existe)
                if n_noise > 0:
                    mask_no_noise = labels != -1
                    X_clean = X[mask_no_noise]
                    labels_clean = labels[mask_no_noise]
                    n_clusters_clean = len(np.unique(labels_clean))
                else:
                    X_clean = X
                    labels_clean = labels
                    n_clusters_clean = n_clusters
                
                # Solo calcular métricas si hay suficientes clusters
                if n_clusters_clean > 1:
                    # silhouette = silhouette_score(X_clean, labels_clean)
                    calinski = calinski_harabasz_score(X_clean, labels_clean)
                    davies_bouldin = davies_bouldin_score(X_clean, labels_clean)
                    
                    # print(f"   🎯 Silhouette Score: {silhouette:.4f}")
                    print(f"   🎯 Calinski-Harabasz: {calinski:.2f}")
                    print(f"   🎯 Davies-Bouldin: {davies_bouldin:.4f}")
                    
                    # Guardar resultados
                    resultados_comparacion[algoritmo] = {
                        'modelo': modelo,
                        'resultados': resultados,
                        'tiempo_total': total_time,
                        'tiempo_entrenamiento': training_time,
                        'tiempo_asignacion': assign_time,
                        'n_clusters': n_clusters,
                        'n_clusters_clean': n_clusters_clean,
                        'n_noise': n_noise,
                        'pct_noise': n_noise/len(resultados)*100,
                        # 'silhouette': silhouette,
                        'calinski_harabasz': calinski,
                        'davies_bouldin': davies_bouldin
                    }
                    
                    # Para tabla comparativa
                    metricas_comparacion.append({
                        'Algoritmo': algoritmo.upper(),
                        'Tiempo_Total': round(total_time, 2),
                        'N_Clusters': n_clusters,
                        'Ruido_%': round(n_noise/len(resultados)*100, 1),
                        # 'Silhouette': round(silhouette, 4),
                        'Calinski_H': round(calinski, 2),
                        'Davies_B': round(davies_bouldin, 4),
                        'Status': '✅'
                    })
                    
                else:
                    print(f"   ⚠️ Insuficientes clusters limpios para métricas")
                    metricas_comparacion.append({
                        'Algoritmo': algoritmo.upper(),
                        'Tiempo_Total': round(total_time, 2),
                        'N_Clusters': n_clusters,
                        'Ruido_%': round(n_noise/len(resultados)*100, 1),
                        'Silhouette': 'N/A',
                        'Calinski_H': 'N/A',
                        'Davies_B': 'N/A',
                        'Status': '⚠️'
                    })
                    
            except Exception as e:
                print(f"   ❌ Error calculando métricas: {str(e)}")
                metricas_comparacion.append({
                    'Algoritmo': algoritmo.upper(),
                    'Tiempo_Total': round(total_time, 2),
                    'N_Clusters': n_clusters,
                    'Ruido_%': round(n_noise/len(resultados)*100, 1) if n_noise > 0 else 0,
                    'Silhouette': 'ERROR',
                    'Calinski_H': 'ERROR',
                    'Davies_B': 'ERROR',
                    'Status': '❌'
                })
                
        else:
            print(f"   ⚠️ Solo 1 cluster generado - algoritmo no útil")
            metricas_comparacion.append({
                'Algoritmo': algoritmo.upper(),
                'Tiempo_Total': round(total_time, 2),
                'N_Clusters': n_clusters,
                'Ruido_%': 0,
                'Silhouette': 'N/A',
                'Calinski_H': 'N/A',
                'Davies_B': 'N/A',
                'Status': '⚠️'
            })
            
    except Exception as e:
        print(f"   ❌ ERROR CRÍTICO con {algoritmo}: {str(e)}")
        metricas_comparacion.append({
            'Algoritmo': algoritmo.upper(),
            'Tiempo_Total': 'FALLÓ',
            'N_Clusters': 'FALLÓ',
            'Ruido_%': 'FALLÓ',
            'Silhouette': 'FALLÓ',
            'Calinski_H': 'FALLÓ',
            'Davies_B': 'FALLÓ',
            'Status': '💥'
        })
        continue

In [ ]:
# =============================================================================
# TABLA COMPARATIVA FINAL
# =============================================================================

print("\n" + "="*100)
print("📊 TABLA COMPARATIVA COMPLETA DE TODOS LOS ALGORITMOS")
print("="*100)

if metricas_comparacion:
    df_comparacion = pd.DataFrame(metricas_comparacion)
    
    # Filtrar solo algoritmos exitosos para ranking
    df_exitosos = df_comparacion[df_comparacion['Status'] == '✅'].copy()
    
    if len(df_exitosos) > 0:
        # Ordenar por Silhouette Score (mayor es mejor)
        df_exitosos = df_exitosos.sort_values('Silhouette', ascending=False)
        
        print("🏆 RANKING DE ALGORITMOS EXITOSOS:")
        print("-" * 50)
        print(df_exitosos.to_string(index=False))
        
        print(f"\n🥇 PODIUM DE GANADORES:")
        print("-" * 30)
        for i, (idx, row) in enumerate(df_exitosos.head(3).iterrows(), 1):
            emoji = "🥇" if i == 1 else "🥈" if i == 2 else "🥉"
            print(f"{emoji} {i}° lugar: {row['Algoritmo']} (Silhouette: {row['Silhouette']})")
            
        # Guardar resultados
        df_comparacion.to_csv('./comparacion_completa_clustering.csv', index=False)
        print(f"\n💾 Resultados completos guardados en: comparacion_completa_clustering.csv")
        
    else:
        print("❌ Ningún algoritmo fue completamente exitoso")
        
    # Mostrar tabla completa (incluyendo errores)
    print(f"\n📋 RESUMEN COMPLETO (incluye errores):")
    print("-" * 40)
    print(df_comparacion.to_string(index=False))
    
else:
    print("💥 No se pudieron ejecutar algoritmos")

print("\n" + "="*100)
print("🎉 COMPARACIÓN COMPLETA DE 9 ALGORITMOS FINALIZADA")
print("="*100)